In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 300
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm

import iridia_af.hyperparameters as hp
import config as cfg
from model import CNNModel, CNNModelConfig
from iridia_af.record import create_record

In [ ]:
# folder_name = "20230701-161104"
folder_name = "20230720-164047"
folder_path = Path(hp.LOG_DL_PATH, folder_name)
assert folder_path.exists()

model_path = Path(folder_path, "model.pt")
assert model_path.exists()
#
config = CNNModelConfig(cfg.WINDOW_SIZE)
model = CNNModel(config)
model.load_state_dict(torch.load(model_path))
model.float()
model.eval()

# folder path
folder_path.absolute()

In [ ]:
metadata_df = pd.read_csv(hp.METADATA_PATH)
record = create_record("record_104", metadata_df, hp.RECORDS_PATH)
record.load_ecg(clean_front=True)

In [ ]:
all_y_pred = [[] for _ in range(len(record.ecg[0]))]
for i in tqdm(range(0, len(record.ecg[0]) - cfg.WINDOW_SIZE, cfg.TESTING_STEP)):
    x = torch.tensor(record.ecg[0][i:i + cfg.WINDOW_SIZE, 0].copy()).float()
    x = x.unsqueeze(0).unsqueeze(0)
    y_pred = model(x)
    for j in range(i, i + cfg.WINDOW_SIZE):
        all_y_pred[j].append(y_pred.detach().numpy()[0][0])

In [ ]:
y_pred = []
for i in tqdm(all_y_pred):
    if len(i) == 0:
        y_pred.append(0)
    else:
        y_pred.append(np.mean(i))

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(20, 10), sharex=True)

ax_index = 0
ax[ax_index].plot(record.ecg[0][:, 0])
ax[ax_index].set_ylabel("ECG Lead I (mV)")
ax_index += 1

ax[ax_index].plot(record.ecg_labels[0])
ax[ax_index].set_ylim(-0.1, 1.1)
ax[ax_index].set_yticks([0, 1])
ax[ax_index].set_yticklabels(["NSR", "AF"])
ax[ax_index].set_ylabel("Annotation")
ax_index += 1

ax[ax_index].plot(y_pred)
ax[ax_index].set_ylim(-0.1, 1.1)
ax[ax_index].set_yticks([0, 1])
ax[ax_index].set_yticklabels(["NSR", "AF"])
ax[ax_index].set_ylabel("Prediction")
ax_index += 1